In [2]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.4 is required in this notebook
# Earlier 2.x versions will mostly work the same, but with a few bugs
import tensorflow as tf
from tensorflow import keras

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(23)
tf.random.set_seed(23)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)


## Tensorflow Common Types and Operations

#### Constants and Operations

In [3]:
tf_array = tf.constant([[1., 2., 3.], [4., 5., 6.], [7., 8., 9.]])
tf_array

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.],
       [7., 8., 9.]], dtype=float32)>

In [4]:
tf_const = tf.constant(31)
tf_const

<tf.Tensor: shape=(), dtype=int32, numpy=31>

In [5]:
tf_array[:,0]

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([1., 4., 7.], dtype=float32)>

In [6]:
tf_array.shape

TensorShape([3, 3])

In [7]:
tf_array + 1

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[ 2.,  3.,  4.],
       [ 5.,  6.,  7.],
       [ 8.,  9., 10.]], dtype=float32)>

In [8]:
tf_array @ tf_array

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[ 30.,  36.,  42.],
       [ 66.,  81.,  96.],
       [102., 126., 150.]], dtype=float32)>

#### Tensforflow and Numpy

In [9]:
tf_const

<tf.Tensor: shape=(), dtype=int32, numpy=31>

In [10]:
tf_const.numpy()

31

In [11]:
tf_array.numpy()

array([[1., 2., 3.],
       [4., 5., 6.],
       [7., 8., 9.]], dtype=float32)

In [12]:
type(tf_array.numpy())

numpy.ndarray

In [13]:
numpy_array = np.array([[1, 1, 2], [3, 5 ,8]])
numpy_array

array([[1, 1, 2],
       [3, 5, 8]])

In [14]:
tf.constant(numpy_array)

<tf.Tensor: shape=(2, 3), dtype=int64, numpy=
array([[1, 1, 2],
       [3, 5, 8]])>

#### Tensorflow Variables

In [15]:
tf_var = tf.Variable([[1., 2., 3.], [4., 5., 6.], [7., 8., 9.]])
tf_var

<tf.Variable 'Variable:0' shape=(3, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.],
       [7., 8., 9.]], dtype=float32)>

In [16]:
tf_var[:, 1].assign(tf_var[:, 1] + tf_var[:, 2])
tf_var

<tf.Variable 'Variable:0' shape=(3, 3) dtype=float32, numpy=
array([[ 1.,  5.,  3.],
       [ 4., 11.,  6.],
       [ 7., 17.,  9.]], dtype=float32)>

#### Tensorflow Strings

In [17]:
tf_str = tf.constant("cheeto")
tf_str

<tf.Tensor: shape=(), dtype=string, numpy=b'cheeto'>

In [18]:
tf_chars = tf.constant([c for c in "cheeto"])
tf_chars

<tf.Tensor: shape=(6,), dtype=string, numpy=array([b'c', b'h', b'e', b'e', b't', b'o'], dtype=object)>

In [19]:
tf.strings.unicode_decode(tf_chars, "UTF-8")

<tf.RaggedTensor [[99],
 [104],
 [101],
 [101],
 [116],
 [111]]>

#### Tensorflow Sets

Each row in a matrix represents a set, so each set is a vector.

In [20]:
tf_set1 = tf.constant([[1, 3, 5, 7], [9, 11, 13, 15]])
tf_set2 = tf.constant([[0, 2, 5], [13, 8, 11]])
tf.sparse.to_dense(tf.sets.union(tf_set1, tf_set2))

<tf.Tensor: shape=(2, 6), dtype=int32, numpy=
array([[ 0,  1,  2,  3,  5,  7],
       [ 8,  9, 11, 13, 15,  0]], dtype=int32)>

## Custom Activation Functions, Initializers, Metrics and Losses

In [21]:
def my_sigmoid(z):
    return 1. / (1. + tf.exp(-z))

In [22]:
my_sigmoid(0.1)

<tf.Tensor: shape=(), dtype=float32, numpy=0.5249792>

In [23]:
def my_glorot_initializer(shape):
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=tf.float32)

In [24]:
glorot_weights = my_glorot_initializer((30, 8))
glorot_weights

<tf.Tensor: shape=(30, 8), dtype=float32, numpy=
array([[ 1.68732136e-01,  2.40755677e-01,  1.40877783e-01,
         2.43688617e-02,  3.40668082e-01, -7.53804110e-03,
         9.01103616e-02, -3.23222548e-01],
       [ 5.29276431e-02, -2.23672464e-01, -2.52694860e-02,
        -2.67078191e-01,  1.88489303e-01,  1.59104183e-01,
        -1.84604842e-02, -6.07359968e-02],
       [-3.84714492e-02,  1.06007385e-03, -2.15167794e-02,
         4.11770493e-01,  2.51709372e-01,  1.22047231e-01,
         1.10198088e-01,  4.17397497e-03],
       [-1.07939593e-01,  7.23771974e-02,  7.54389912e-02,
         1.43001243e-01, -1.62578464e-01, -3.17653179e-01,
        -2.39377171e-01, -2.72420883e-01],
       [-1.19647287e-01,  1.79983184e-01,  3.22413266e-01,
        -5.98094881e-01,  9.79093611e-02, -1.56773906e-02,
        -1.16528822e-02, -2.00743720e-01],
       [ 3.96184713e-01, -1.07933037e-01, -1.54256955e-01,
         2.51009524e-01,  2.11574491e-02, -1.18471429e-01,
        -1.72067761e-01, -3.

In [25]:
def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

In [26]:
my_l1_regularizer(glorot_weights)

<tf.Tensor: shape=(), dtype=float32, numpy=0.4405663>

In [27]:
def my_l2_regularizer(weights):
    return tf.reduce_sum(0.01 * tf.square(weights))

In [28]:
my_l2_regularizer(glorot_weights)

<tf.Tensor: shape=(), dtype=float32, numpy=0.12027138>

In [29]:
def my_relu(z):
    return tf.where(z < 0., tf.zeros_like(z), z)

In [30]:
my_relu(-3)

<tf.Tensor: shape=(), dtype=int32, numpy=0>

In [31]:
my_relu(1)

<tf.Tensor: shape=(), dtype=int32, numpy=1>

In [32]:
layer = keras.layers.Dense(30, activation=my_relu, 
                           kernel_initializer=my_glorot_initializer,
                           kernel_regularizer=my_l1_regularizer)

In case we have hyperparameters, we need to define a class

In [33]:
class MyL1Regularizer(keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor
    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))
    def get_config(self):
        return {"factor": self.factor}

* For a custom metric we need to inherit from `keras.metrics.Metric`.

* For a custom loss function we need to inherit from `keras.losses.Loss`.

## Custom Layers

In [34]:
class MyDense(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)

    def build(self, batch_input_shape):
        self.kernel = self.add_weight(
            name="kernel", shape=[batch_input_shape[-1], self.units],
            initializer="glorot_normal")
        self.bias = self.add_weight(
            name="bias", shape=[self.units], initializer="zeros")
        super().build(batch_input_shape) # must be at the end

    def call(self, X):
        return self.activation(X @ self.kernel + self.bias)

    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units])

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "units": self.units,
                "activation": keras.activations.serialize(self.activation)}

## Data Processing

In [35]:
np.random.seed(42)
tf.random.set_seed(42)

n_steps = 5
dataset = tf.data.Dataset.from_tensor_slices(tf.range(15))
dataset = dataset.window(n_steps, shift=2, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(n_steps))
dataset = dataset.shuffle(10).map(lambda window: (window[:-1], window[1:]))
dataset = dataset.batch(3).prefetch(1)
for index, (X_batch, Y_batch) in enumerate(dataset):
    print("_" * 20, "Batch", index, "\nX_batch")
    print(X_batch.numpy())
    print("=" * 5, "\nY_batch")
    print(Y_batch.numpy())

____________________ Batch 0 
X_batch
[[6 7 8 9]
 [2 3 4 5]
 [4 5 6 7]]
===== 
Y_batch
[[ 7  8  9 10]
 [ 3  4  5  6]
 [ 5  6  7  8]]
____________________ Batch 1 
X_batch
[[ 0  1  2  3]
 [ 8  9 10 11]
 [10 11 12 13]]
===== 
Y_batch
[[ 1  2  3  4]
 [ 9 10 11 12]
 [11 12 13 14]]


2023-06-09 11:39:28.595351: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


#### Exercise

Change the previous cell to have as the target variable, Y, only the last element of the window, while the first `n_steps - 1` elements are assigned to X 

## Encoding using One-Hot Encoder

In [44]:
vocab = ["CHEETO", "SPURS", "ATLAS", "FEDERER", "JORDAN"];
indices = tf.range(len(vocab), dtype=tf.int64)
indices

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([0, 1, 2, 3, 4])>

In [45]:
table_init = tf.lookup.KeyValueTensorInitializer(vocab, indices)
num_oov_buckets = 2
table = tf.lookup.StaticVocabularyTable(table_init, num_oov_buckets)
table

In [48]:
categories = tf.constant(["CHEETO", "MAGIC", "FEDERER", "JORDAN", "MAGIC", "JOKIC", "JORCH", "SPURS"])
cat_indices = table.lookup(categories)
cat_indices

<tf.Tensor: shape=(8,), dtype=int64, numpy=array([0, 6, 3, 4, 6, 6, 5, 1])>

In [49]:
cat_one_hot = tf.one_hot(cat_indices, depth=len(vocab) + num_oov_buckets)
cat_one_hot

<tf.Tensor: shape=(8, 7), dtype=float32, numpy=
array([[1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0.]], dtype=float32)>

## Encoding using Embeddings

In [52]:
embedding_dim = 2
embed_init = tf.random.uniform([len(vocab) + num_oov_buckets, embedding_dim])
embedding_matrix = tf.Variable(embed_init)
embedding_matrix

<tf.Variable 'Variable:0' shape=(7, 2) dtype=float32, numpy=
array([[0.7413678 , 0.62854624],
       [0.01738465, 0.3431449 ],
       [0.51063764, 0.3777541 ],
       [0.07321596, 0.02137029],
       [0.2871771 , 0.4710616 ],
       [0.6936141 , 0.07321334],
       [0.93251204, 0.20843053]], dtype=float32)>

In [53]:
categories = tf.constant(["CHEETO", "MAGIC", "FEDERER", "JORDAN", "MAGIC", "JOKIC", "JORCH", "SPURS"])
cat_indices = table.lookup(categories)
cat_indices

<tf.Tensor: shape=(8,), dtype=int64, numpy=array([0, 6, 3, 4, 6, 6, 5, 1])>

In [54]:
tf.nn.embedding_lookup(embedding_matrix, cat_indices)

<tf.Tensor: shape=(8, 2), dtype=float32, numpy=
array([[0.7413678 , 0.62854624],
       [0.93251204, 0.20843053],
       [0.07321596, 0.02137029],
       [0.2871771 , 0.4710616 ],
       [0.93251204, 0.20843053],
       [0.93251204, 0.20843053],
       [0.6936141 , 0.07321334],
       [0.01738465, 0.3431449 ]], dtype=float32)>

In [56]:
embedding = keras.layers.Embedding(input_dim=len(vocab) + num_oov_buckets, 
                                   output_dim=embedding_dim)
embedding(cat_indices)

<tf.Tensor: shape=(8, 2), dtype=float32, numpy=
array([[-0.00202948,  0.00609867],
       [ 0.00915229, -0.01083909],
       [-0.0110005 ,  0.00738021],
       [ 0.03156973, -0.00937718],
       [ 0.00915229, -0.01083909],
       [ 0.00915229, -0.01083909],
       [ 0.03935714,  0.02117929],
       [ 0.03416841, -0.04140834]], dtype=float32)>